In [10]:
# import os
# import pandas as pd

# BASE_DIR = "../llama_profiling/experiments/llama_profiling"
# OUTPUT_FILE = "merged_energibridge.csv"

# csv_files = [
#     os.path.join(root, f)
#     for root, _, files in os.walk(BASE_DIR)
#     for f in files if f.endswith(".csv")
# ]

# dfs = []
# for path in csv_files:
#     df = pd.read_csv(path)
#     df.columns = df.columns.str.strip()
#     dfs.append(df)

# merged = pd.concat(dfs, ignore_index=True)

# merged.to_csv(OUTPUT_FILE, index=False)
# print(f"Merged CSV saved: {OUTPUT_FILE}")

import os
import pandas as pd
import re
import numpy as np

In [18]:
BASE_DIR = "../llama_profiling/experiments/llama_profiling"
OUTPUT_FILE = "merged_energibridge.csv"

csv_files = [
    os.path.join(root, f)
    for root, _, files in os.walk(BASE_DIR)
    for f in files if f.endswith(".csv")
][1:20]

dfs = []

results = []


def split_run_id(val):
    match = re.match(r'(?:run_?|)(\d+)_+(repetition_\d+)', val)
    if match:
        run_num, repetition = match.groups()
        return f"run{run_num}", repetition
    else:
        parts = val.split("_", 1)
        run = parts[0] if parts else None
        rep = parts[1] if len(parts) > 1 else None
        return run, rep
    
for i,path in enumerate(csv_files):
    folder = path[:path.rfind("\\")]
    #if 'run_0_repetition_15' not in path:
    #    print('ignoring', path)
    #    continue

    print('running', path)
    df = pd.read_csv(path)
    df.columns = df.columns.str.strip()
        
    # Fix overflow and normalize to energy_diff
    diff = df["PACKAGE_ENERGY (J)"] - df["PACKAGE_ENERGY (J)"].shift(1)
    df["energy_diff"] = np.where(diff < 0, df["PACKAGE_ENERGY (J)"], diff.fillna(0))
    
    # Calculate GPU consumption
    df["gpu_joules"] = df["GPU0_POWER (mWatts)"] / 5000 + df["energy_diff"]

    # Display all rows
    #with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    #    print(df[["gpu_joules", "PACKAGE_ENERGY (J)"]], )
        
    # Total consumption
    energy_consumption_total=sum(df["gpu_joules"]) + sum(df["energy_diff"])
    
    # Merge .tsv data
    tsv_df = pd.read_csv(folder+"\\prompts_out.tsv", sep="\t")
    
    tsv_df['prompt_length'] = tsv_df["prompt"].str.len()
    tsv_df['response_length'] = tsv_df["response"].str.len()
    
    timestamp_start_prompts, timestamp_end_prompts = tsv_df.iloc[0]['start'], tsv_df.iloc[-1]['end']

    df = df[(df['Time'] >= timestamp_start_prompts) & (df['Time'] <= timestamp_end_prompts)]
    
    print(df)
    
    # Reduced data from the .csv and .tsv        
    print('response length sum (characters): ', sum(tsv_df['response_length']))
    print('energy (joules): ', energy_consumption_total)
    print('duration (ms)', df.iloc[-1]['Time']-df.iloc[0]['Time'])
    print('accuracy', 30)
    break
    continue

    


running ../llama_profiling/experiments/llama_profiling\run_0_repetition_0\energibridge.csv
Empty DataFrame
Columns: [Delta, Time, CPU_FREQUENCY_0, CPU_FREQUENCY_1, CPU_FREQUENCY_10, CPU_FREQUENCY_11, CPU_FREQUENCY_12, CPU_FREQUENCY_13, CPU_FREQUENCY_14, CPU_FREQUENCY_15, CPU_FREQUENCY_16, CPU_FREQUENCY_17, CPU_FREQUENCY_18, CPU_FREQUENCY_19, CPU_FREQUENCY_2, CPU_FREQUENCY_20, CPU_FREQUENCY_21, CPU_FREQUENCY_22, CPU_FREQUENCY_23, CPU_FREQUENCY_24, CPU_FREQUENCY_25, CPU_FREQUENCY_26, CPU_FREQUENCY_27, CPU_FREQUENCY_28, CPU_FREQUENCY_29, CPU_FREQUENCY_3, CPU_FREQUENCY_30, CPU_FREQUENCY_31, CPU_FREQUENCY_4, CPU_FREQUENCY_5, CPU_FREQUENCY_6, CPU_FREQUENCY_7, CPU_FREQUENCY_8, CPU_FREQUENCY_9, CPU_USAGE_0, CPU_USAGE_1, CPU_USAGE_10, CPU_USAGE_11, CPU_USAGE_12, CPU_USAGE_13, CPU_USAGE_14, CPU_USAGE_15, CPU_USAGE_16, CPU_USAGE_17, CPU_USAGE_18, CPU_USAGE_19, CPU_USAGE_2, CPU_USAGE_20, CPU_USAGE_21, CPU_USAGE_22, CPU_USAGE_23, CPU_USAGE_24, CPU_USAGE_25, CPU_USAGE_26, CPU_USAGE_27, CPU_USAGE_28,

IndexError: single positional indexer is out-of-bounds

In [8]:
dff = tsv_df[0:1]

In [9]:
dff

,start,end,prompt,response,prompt_length,response_length
0,1760502188,1760502190,Passage: Persian language -- Persian (/ˈpɜːrʒə...,"Based on the given passage, it can be said t...",696,799
